<a href="https://colab.research.google.com/github/sunnygong369/Covisitation-Matrix/blob/main/MOVIE_MATRIX_COMPLETE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Settings**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


*   **Unpacking the zip to df**
(time: 15s)

**Data source:** https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset?select=ratings.csv



In [ ]:
import zipfile
import pandas as pd

df_zip = zipfile.ZipFile('/content/drive/MyDrive/Colab Notebooks/ratings.csv.zip')
df = pd.read_csv(df_zip.open('ratings.csv'))

df

*   **Taking 1/5 of the df**

In [21]:
end_row = int(len(df.index)*(1/5))


In [22]:
import pandas as pd
import numpy as np

#df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movie_ratings.csv') #load the csv file --> the "history table"

df = df[["userId", "movieId", "timestamp"]]
df = df.reset_index(drop = True)
df = df[:end_row]
df


,userId,movieId,timestamp
0,1,110,1425941529
1,1,147,1425942435
2,1,858,1425941523
3,1,1221,1425941546
4,1,1246,1425941556
...,...,...,...
1040966,10595,2600,1457384703
1040967,10595,2617,1456091165
1040968,10595,2628,1458943173
1040969,10595,2692,1492975191


### **Rearranging the dataset**

*   **Sorting by timestamp [to account for order]**
(time: 21s)

In [ ]:
df_grp = df.groupby('userId') #group by "userId"

df_grp_srt = df_grp.apply(lambda x: x.sort_values('timestamp')) #sorting each group by timestamp (ordered)

df_grp_srt

In [ ]:
#quick check on timestamp sorting:
df_grp_srt[df_grp_srt['userId'] == 99] #replace 99 with any number

*   **Filtering out the timestamp column**
(time: 2s)

In [25]:
df_new = zip(df_grp_srt['userId'].tolist(),df_grp_srt['movieId'].tolist()) #put 'userId' and 'movieId' to list
df_new = pd.DataFrame(df_new)
df_new.columns = ['User IDs', 'Movie IDs'] #rename the columns
df_new.index.name = 'Index'

df_new

,User ID,Movie ID
Index,,
0,1,2762
1,1,54503
2,1,112552
3,1,96821
4,1,5577
...,...,...
1040966,10595,2355
1040967,10595,455
1040968,10595,500


------------------------------------------------------------------------------
*   **Step abandoned [crashed for trying to display entries in "user1 item1 item2 item3 ... itemN" format]**
------------------------------------------------------------------------------


In [8]:
# aimed to arrange data in the format as such: "user1 item1 item2 item3 ... itemN" for all the rows
#it will crash --> don't run
#it will crash --> don't run
#it will crash --> don't run

df_new_wide = df_new.pivot_table(index='User IDs', columns='Index', values='Movie IDs', aggfunc='first') 

df_new_wide

<ipython-input-8-b49dab7d378b>:6: PerformanceWarning: The following operation may generate 279047998341 cells in the resulting pandas object.
  df_new_wide = df_new.pivot_table(index='User ID', columns='Index', values='Movie ID', aggfunc='first')


KeyboardInterrupt: ignored

*   **Rearranging the entries**

In [ ]:
df_new1 = df_new.groupby('User IDs')['Movie IDs'].agg(list).reset_index() #take the "User ID" and "Movie ID" columns to create the users' history table
df_new1

In [ ]:
df_new2 = df_new1["Movie IDs"] #select "Movie ID" --> to creat pairs at the next stage
df_new2

*   **Finding all the pairs from the entries** (time: 10s + 3s)

In [ ]:
from nltk import bigrams
bigrams_list = [list(bigrams(i)) for i in df_new2] #finding all the pairs
bigrams_list

bigrams_list = [[str(i) for i in sublist] for sublist in bigrams_list] #making all the pairs to string (for counting purposes)
bigrams_list

In [ ]:
bigrams_list = [[i.replace('(', '').replace(')', '') for i in sublist] for sublist in bigrams_list] #remove all brackets
bigrams_list

*   **Counting all the pairs** (time: 3s)

In [ ]:
from collections import Counter

all_pairs = [i for sublist in bigrams_list for i in sublist] #list of lists unpacked

pairs_counter = Counter(all_pairs) #counting unique pairs from "history" (i.e. ordered entries)
pairs_counter

### **Constructing a [blank] Covisitation Matrix**

*   **Unique IDs for labels**

In [ ]:
unique_val = pd.DataFrame(pd.unique(df_new2.explode())) #unpack and collect unique values -> then make it a df
unique_val_srt = unique_val[0].sort_values().astype(str).tolist() #sorting df by unique values -> making them strings (for labeling purposes) in list
unique_val_srt

*   **Blank matrix filled with zeros**

In [ ]:
co_matrix = np.zeros((len(unique_val_srt), len(unique_val_srt)), dtype=int) #a blank covisitation matrix created
co_matrix = pd.DataFrame(co_matrix) #converting matrix to df
co_matrix.index = unique_val_srt #labeling the rows with unique IDs
co_matrix.columns = unique_val_srt #labeling the columns ...
co_matrix

------------------------------------------------------------------------------
*   **Step abandoned [crashed for trying to display all the pairs in matrix]**
------------------------------------------------------------------------------


In [ ]:
#display all the pairs in the matrix
#don't run --> it will crash
#don't run --> it will crash
#don't run --> it will crash

for i in co_matrix.index: 
  for j in co_matrix.columns:
    co_matrix[i][j] = (str(i)+str(j))

co_matrix

### **Constructing the Covisitation Matrix** 
**Filling the Covisitation Matrix** (time: 2m36s)

In [ ]:
for key, value in dict(pairs_counter).items():
   a = key.split(",")[0].strip()
   b = key.split(",")[1].split(":")[0].strip()
   c = value

   co_matrix.loc[a,b] = c ###### it may have writen c to cell [a, b], but it is not showing in the table below

co_matrix


In [ ]:
#for checking only (i.e. the top 50x50 matrix)
co_matrix[:50][:50]

In [ ]:
#for checking only (i.e. checking the values of a, b, c) 

for key, value in dict(pairs_counter).items():
   a = key.split(",")[0].strip()
   b = key.split(",")[1].split(":")[0].strip()
   c = value

   print (a, b, c)

### **Predicting**

*   **Normalizing the matrix (could be skipped)** (time: long)




In [ ]:
from sklearn.preprocessing import normalize #TO CONFIRM (How it should be normalized?) ALSO, cosine_similarity with or without the normalizing the matrix seem to give the same result

co_matrix_norm = normalize(co_matrix)
co_matrix_norm.index = unique_val_srt #label the rows with the list of unique item ID
co_matrix_norm.columns = unique_val_srt #label the columns ...

co_matrix_norm

*   **Cosine_similarity_scores (with normalized matrix)** (time: long)







In [ ]:
from sklearn.metrics.pairwise import cosine_similarity #[cosine_similarity (with normalized matrix)]

cos_score_df = pd.DataFrame(cosine_similarity(co_matrix_norm)) #compute similarity score with cosine_similarity (without normalized the matrix)
cos_score_df.index = unique_val_srt #label the rows with the list of unique item ID
cos_score_df.columns = unique_val_srt #label the columns ...

cos_score_df

In [196]:
new_user = np.zeros(len(co_matrix), dtype=int) #create a vector as a new user entry

for i in range(100): #generate 1 for 1000 random indices
  new_user[np.random.randint(0, len(co_matrix))]=1 

len(new_user) == len(co_matrix)
new_user

array([0, 0, 0, ..., 0, 0, 0])

In [201]:
#testing
A = new_user
B = co_matrix.to_numpy() #can use co_matrix directly here

out = np.matmul(A, B)
out

array([14,  4,  7, ...,  0,  0,  0])

In [ ]:
aa = pd.DataFrame(zip(unique_val_srt, out, new_user))
aa.sort_values(1,ascending=False)


*   **Predicting (making recommendations) for a new user**

In [211]:
A = new_user
B = co_matrix.to_numpy() #can use co_matrix directly here

out = np.matmul(A, B)
out_df = pd.DataFrame(zip(unique_val_srt, out, new_user), columns = ['Movie IDs','Scores', 'New User Entries']) #combine 'row labels', 'scores', 'new user entries' 
candidates = out_df.sort_values('Scores', ascending=False) #sorting by scores for top N candidates ('Entries' = 1 indicates user's 'implied preference')
candidates


,Movie IDs,Scores,New User Entries
585,597,122,0
530,539,93,0
33,34,72,0
356,364,68,0
581,593,60,0
...,...,...,...
7978,8989,0,0
7979,8991,0,0
7980,8992,0,0
7981,8994,0,0
